In [1]:
import ee
import geemap
ee.Initialize()

In [2]:
Map = geemap.Map()

In [3]:
#AOI Casa-Rabat

AOI=ee.Geometry.Rectangle([-7.8,34.5,-6.1,33.3])

image = ee.ImageCollection("COPERNICUS/S2") \
    .filterBounds(AOI) \
    .filterDate('2020-01-01', '2020-10-31') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) 
    
img = ee.Image(image.median()).clip(AOI)


vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2']
}

Map.centerObject(AOI,8)
Map.addLayer(img, vis_params, "sentinel2")

In [4]:
training = img.sample(**{
    'region': AOI,
    'scale': 200,
    'numPixels': 5000,
    'seed': 0,
    'geometries': True  
})

Map.addLayer(training, {}, 'training', False)

In [5]:
# Instantiate the clusterer and train it.
n_clusters = 6
clusterer = ee.Clusterer.wekaKMeans(n_clusters).train(training)

# Cluster the input using the trained clusterer.
result = img.cluster(clusterer)

# # Display the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, 'clusters')

In [6]:
Map

Map(center=[33.90151095239523, -6.950000000000058], controls=(WidgetControl(options=['position'], widget=HBox(…

In [7]:
legend_keys = ['intensive agriculture/vegetation','buildings/ dense habitats','water','crop production/forest',  'houses/extensive agriculture/clair forest' ,'reforestation/bare soil']
legend_colors = ['#BEBADA','#000000','#0000FF','#00FF00','#FB8072','#FF0000']

#Reclassify the map

result = result.remap([0, 1, 2, 3, 4,5], [1, 2, 3, 4, 5,6])

Map.addLayer(result, {'min': 1, 'max': 6, 'palette': legend_colors}, 'Labelled clusters')
Map.add_legend(legend_keys=legend_keys, legend_colors=legend_colors, position='bottomright')


In [8]:
print('Change layer opacity:')
cluster_layer = Map.layers[-1]
cluster_layer.interact(opacity=(0, 1, 0.1))

Change layer opacity:


Box(children=(FloatSlider(value=1.0, description='opacity', max=1.0),))

In [11]:
import os
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
out_file = os.path.join(out_dir, 'CASA-RABAT.tif')
geemap.ee_export_image(result, filename=out_file, scale=90)

Generating URL ...
Please wait ...
An error occurred while downloading.
